In [ ]:
#adjust total xp variable to substract hits

total_xp = round(so.expr_sum((lineup[p,w] + captain[p,w]) * points_player_week[p,w] for p in players for w in gameweeks).get_value()+
                     (so.expr_sum(penalized_transfers[w] for w in gameweeks).get_value()*-4),1)

#after you run the solver with iterations on chips (or any iterations for that matter) just execute this 

summaries=[]

for i in range(options['iteration']):
    res_str=str(result[i]['summary'])

    buy=re.findall(r'\Buy (.*?)\nS',res_str,re.DOTALL)
    buy_clean = [s.replace('\nBuy', ',') for s in buy]

    sell=re.findall(r'\Sell (.*?)\n-',res_str,re.DOTALL)
    sell_clean = [s.replace('\nSell', ',') for s in sell]

    gw=re.findall('GW (.*?)\nLineup',res_str,re.DOTALL)
    gw_clean=[s[:2] for s in gw if 'Buy' in s]

    chip=re.findall(r'\nCHIP (.*?)\nitb',res_str,re.DOTALL)
    chip_clean=[s[:2] for s in gw if 'FH' in s or 'BB' in s or 'WC' in s]

    hit=re.findall(r'hits=(.*?)\n',res_str,re.DOTALL)

    gw_all=re.findall('GW (.*?)\nLineup',res_str,re.DOTALL)
    all_gw=pd.DataFrame([s[:2] for s in gw_all])
    all_gw.columns = ['GW']

    transfers = pd.DataFrame(list(zip(sell_clean,buy_clean)), index=gw_clean, columns =['Sell',"Buy"])
    chips=pd.DataFrame(chip, index=chip_clean, columns =['chip'])
    hits=pd.DataFrame(hit, index=all_gw.GW, columns =['hits'])
    summary = pd.merge(all_gw, transfers,how='left',left_on='GW', right_index=True)
    summary = pd.merge(summary, chips,how='left',left_on='GW', right_index=True)
    summary = pd.merge(summary, hits,how='left',left_on='GW', right_index=True)

    summary.fillna('None',inplace=True)
    
    summary['iteration']=i+1
    summaries.append(summary)

for o in range(options['iteration']):
    print(f"Solution #{summaries[o]['iteration'][0]} (EV:{round(result[o]['total_xp'],2)} Score:{round(result[o]['score'],2)})")
    for i in range(len(all_gw)):
        if summaries[o]['Sell'][i]=='None' and summaries[o]['chip'][i]=='None':
            print(f"GW{summaries[o]['GW'][i]}: Save FT ")
        elif summaries[o]['Sell'][i]!='None' and summaries[o]['chip'][i] in ['BB','TC']:
            print(f"GW{summaries[o]['GW'][i]}: CHIP:|{summaries[o]['chip'][i]}|, {summaries[o]['Sell'][i]} --> {summaries[o]['Buy'][i]}")
        elif summaries[o]['Sell'][i]=='None' and summaries[o]['chip'][i] in ['BB','TC']:
            print(f"GW{summaries[o]['GW'][i]}: CHIP:|{summaries[o]['chip'][i]}|,  Save FT")
        elif summaries[o]['chip'][i]!='None':
            print(f"GW{summaries[o]['GW'][i]}: CHIP:|{summaries[o]['chip'][i]}|")
        elif summaries[o]['hits'][i]=='0':
            print(f"GW{summaries[o]['GW'][i]}: {summaries[o]['Sell'][i]} --> {summaries[o]['Buy'][i]}")
        else:
            print(f"GW{summaries[o]['GW'][i]}: {summaries[o]['Sell'][i]} --> {summaries[o]['Buy'][i]} {summaries[o]['hits'][i]}")
    #print('\n')
    print('-----------------------------------------\n')